In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
import tensorflow as tf

In [2]:
# Function to average word vectors for a text
def average_word_vectors(words, model, num_features):
    feature_vec = np.zeros((num_features,), dtype="float32")
    n_words = 0

    for word in words:
        if word in model.wv:
            n_words += 1
            feature_vec = np.add(feature_vec, model.wv[word])

    if n_words > 0:
        feature_vec = np.divide(feature_vec, n_words)

    return feature_vec

In [3]:
df = pd.read_csv("preprocessed_sentiment140.csv")
w2v = Word2Vec.load("word2vec.model")

In [4]:
df_train, df_test = train_test_split(df, test_size=1-0.8, random_state=42)
print("TRAIN size:", len(df_train))
print("TEST size:", len(df_test))

TRAIN size: 1280000
TEST size: 320000


In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_train.text)

vocab_size = len(tokenizer.word_index) + 1
print("Total words", vocab_size)

Total words 594849


In [6]:
x_train = pad_sequences(tokenizer.texts_to_sequences(df_train.text), maxlen=300)
x_test = pad_sequences(tokenizer.texts_to_sequences(df_test.text), maxlen=300)

In [7]:
encoder = LabelEncoder()
encoder.fit(df_train.target.tolist())

y_train = encoder.transform(df_train.target.tolist())
y_test = encoder.transform(df_test.target.tolist())

y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

print("y_train",y_train.shape)
print("y_test",y_test.shape)

y_train (1280000, 1)
y_test (320000, 1)


In [8]:
print("x_train", x_train.shape)
print("y_train", y_train.shape)
print()
print("x_test", x_test.shape)
print("y_test", y_test.shape)

x_train (1280000, 300)
y_train (1280000, 1)

x_test (320000, 300)
y_test (320000, 1)


In [9]:
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in tokenizer.word_index.items():
  if word in w2v.wv:
    embedding_matrix[i] = w2v.wv[word]
print(embedding_matrix.shape)

(594849, 300)


In [10]:
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=300, trainable=False)

In [11]:
model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.5))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 300)          178454700 
                                                                 
 dropout (Dropout)           (None, 300, 300)          0         
                                                                 
 lstm (LSTM)                 (None, 100)               160400    
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 178615201 (681.36 MB)
Trainable params: 160501 (626.96 KB)
Non-trainable params: 178454700 (680.75 MB)
_________________________________________________________________


In [13]:
callbacks = [ ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
              EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=5)]

In [14]:
history = model.fit(x_train, y_train,
                             batch_size=1024,
                             epochs=8,
                             validation_split=0.1,
                             verbose=1,
                             callbacks=callbacks)

Epoch 1/8
1125/1125 [==============================] - 32749s 29s/step - loss: 0.5249 - accuracy: 0.7363 - val_loss: 0.4699 - val_accuracy: 0.7783 - lr: 0.0010
Epoch 2/8
1125/1125 [==============================] - 38862s 35s/step - loss: 0.4853 - accuracy: 0.7644 - val_loss: 0.4607 - val_accuracy: 0.7848 - lr: 0.0010
Epoch 3/8
1125/1125 [==============================] - 40328s 36s/step - loss: 0.4758 - accuracy: 0.7703 - val_loss: 0.4532 - val_accuracy: 0.7890 - lr: 0.0010
Epoch 4/8
 538/1125 [=============>................] - ETA: 5:36:38 - loss: 0.4709 - accuracy: 0.7733